In [131]:
import pandas as pd
import re
import pandas as pd
import csv

pod_mappings = {
    'M.280-SSD-256GB-PCIe44-TLC5WT-T': 'M.280-SSD-256GB-PCIe44-TLC5WT-TD',
    'M.280-SSD-512GB-PCIe44-TLC5WT-T': 'M.280-SSD-512GB-PCIe44-TLC5WT-TD',
    'M.242-SSD-256GB-PCIe34-TLC5WT-T': 'M.242-SSD-256GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-512GB-PCIe34-TLC5WT-T': 'M.242-SSD-512GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-128GB-PCIe34-TLC5WT-T': 'M.242-SSD-128GB-PCIe34-TLC5WT-TD',
    'Cblkit-FP-NRU-230V-AWP_NRU-240S': 'Cblkit-FP-NRU-230V-AWP_NRU-240S-AWP',
}

NAV_mappings = {
    'GC-Jetson-AGX64GB-Orin-Industrial-Nvidia-JetPack-6.0': 'GC-J-A64GB-O-Industrial-Nvidia',
    'GC-Jetson-AGX64GB-Orin-Nvidia-JetPack-6.0': 'GC-Jetson-AGX64GB-Orin-Nvidia',
    'AccsyBx-Cardholder-10108GC-5080_70_70Ti': 'AccsyBx-Cardholder-10108GC-5080',
    'Extnd-mPCIeHS_GPS-M800_Mod_Cbl-40CM_kits': 'E-mPCIe-GPS-M800_Mod_40CM',
    'Cbl-M12A5F-OT2-Black-Red-Fuse-100CM': 'Cbl-M12A5F-OT2-B-Red-Fuse-100CM',
    'AccsyBx-Cardholder-9160GC-2000EAda': 'AccsyBx-Cardholder-9160GC-2000E',
    'M.280-SSD-4TB-PCIe4-TLCWT5-NH-IK': 'M.280-SSD-4TB-PCIe4-TLCWT5NH-IK',
    'M.242-SSD-128GB-PCIe34-TLC5WT-TD': 'M.242-SSD-128GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-256GB-PCIe34-TLC5WT-TD': 'M.242-SSD-256GB-PCIe34-TLC5WT-TD',
    'M.280-SSD-256GB-PCIe44-TLC5WT-TD': 'M.280-SSD-256GB-PCIe44-TLC5WT-TD',
    'M.280-SSD-512GB-PCIe44-TLC5WT-TD': 'M.280-SSD-512GB-PCIe44-TLC5WT-TD',
    'Extnd-mPCIeHS-BTWifi-WT-6218_Mod_Cbl-40CM_kits': 'E-mPCIe-BTWifi-WT-6218_Mod_40CM',
    'GC-Jetson-NX16G-Orin-Nvidia-JetPack6.0': 'GC-Jetson-NX16G-Orin-Nvidia',
    'FPnl-3Ant-NRU-170-PPCseries': 'FPnl-3Ant-NRU-170-PPC series',
}

#"POD"
pod = pd.read_csv("open purchase orders.csv", encoding="utf-8", encoding_errors="replace")
pod.drop(columns=['Name', 'Amount', 'Open Balance', "Rcv'd", "Qty", "Memo"], inplace=True)
pod.rename(columns={"Date": "Order Date", "Num": "QB Num", "Source Name": "Name", "Backordered": "Qty(+)"}, inplace=True)
pod.drop(pod.columns[0], axis=1, inplace=True)
pod.dropna(how='all', inplace=True)
pod.dropna(thresh=5, inplace=True) #刪除有效值少於5個的行
# print(pod['Item'])
pod['Item'] = pod['Item'].str.split(':').str[1]
pod = pod[pod['Item'] != 'Engineer Service- COS']
pod = pod[pod['Item'] != 'RMA Services']
pod['Item'] = pod['Item'].replace(pod_mappings) # 將POD的Item重新命名
# print(pod['QB Num'])
pod['QB Num'] = pod['QB Num'].str.split('(').str[0]
for col in ['Order Date', 'Deliv Date']:
    pod[col] = pd.to_datetime(pod[col]).dt.strftime('%Y/%m/%d')
pod.to_csv('open purchase2.csv', index=False)


#"NAV"
NAV = pd.read_csv("Sales Date return platform.csv", usecols=['Document No.', "Customer PO No.", "Customer Ordering Model",
                                                             "OP Estimated Shipping Date", "Quantity", "No.",
                                                             "Customer Ordering Desc."], encoding='utf-8')
NAV.rename(columns={"Customer PO No.": "QB Num", "Customer Ordering Model": "Item", 'Document No.': "Remark",
                    "OP Estimated Shipping Date": "Ship Date", "Quantity": "Qty(+)"}, inplace=True)
NAV = NAV[NAV['Item'] != 'Engineer Service- COS']
NAV = NAV[NAV['Item'] != 'CUSTOMER SERVICES']
NAV = NAV[NAV['Item'] != 'FORWARDING CHARGE, EXCLUDING IMPORT DUTY.']
# print(NAV['QB Num'])
NAV['QB Num'] = NAV['QB Num'].str.split('(').str[0]

NAV.to_csv('NAV1.csv', index=False)


In [132]:
# Data Type Analysis for POD and NAV
import pandas as pd
import numpy as np

print("=== POD DATA ANALYSIS ===")
print()

# Load POD data
try:
    pod = pd.read_csv('open purchase2.csv')
    print("POD Data Shape:", pod.shape)
    print("\nPOD Columns:")
    print(pod.columns.tolist())
    print("\nPOD Data Types:")
    print(pod.dtypes)
    print("\nPOD First 3 rows:")
    print(pod.head(3))
    # print("\nPOD Sample QB Num values:")
    # print(pod['QB Num'].head(10).tolist())
    # print("\nPOD Sample Item values:")
    # print(pod['Item'].head(10).tolist())
except FileNotFoundError:
    print("POD file not found - 'open purchase2.csv'")
except Exception as e:
    print(f"Error loading POD: {e}")


=== POD DATA ANALYSIS ===

POD Data Shape: (459, 8)

POD Columns:
['Order Date', 'QB Num', 'P. O. #', 'Name', 'Deliv Date', 'Qty(+)', 'Item', 'Inventory Site']

POD Data Types:
Order Date         object
QB Num             object
P. O. #            object
Name               object
Deliv Date         object
Qty(+)            float64
Item               object
Inventory Site     object
dtype: object

POD First 3 rows:
   Order Date      QB Num      P. O. #                        Name  \
0  2025/07/24  POD-251046  SO-20251050  Neousys Technology Incorp.   
1  2025/07/24  POD-251047  SO-20251051  Neousys Technology Incorp.   
2  2025/07/24  POD-251048  SO-20251052  Neousys Technology Incorp.   

   Deliv Date  Qty(+)                        Item Inventory Site  
0  2025/10/03    34.0  AccsyBx-6AntiVG-POC-551VTC      Drop Ship  
1  2025/11/21    24.0  AccsyBx-6AntiVG-POC-551VTC      Drop Ship  
2  2025/12/19    29.0  AccsyBx-6AntiVG-POC-551VTC      Drop Ship  


In [133]:
print("=== NAV DATA ANALYSIS ===")
print()

# Load NAV data
try:
    nav = pd.read_csv('NAV1.csv')
    print("NAV Data Shape:", nav.shape)
    print("\nNAV Columns:")
    print(nav.columns.tolist())
    print("\nNAV Data Types:")
    print(nav.dtypes)
    print("\nNAV First 3 rows:")
    print(nav.head(3))
    # print("\nNAV Sample QB Num values:")
    # print(nav['QB Num'].head(10).tolist())
    # print("\nNAV Sample Item values:")
    # print(nav['Item'].head(10).tolist())
    # print("\nNAV Sample Ship Date values:")
    # print(nav['Ship Date'].head(10).tolist())
    # print("\nNAV Sample No. values:")
    # print(nav['No.'].head(10).tolist())
except FileNotFoundError:
    print("NAV file not found - 'NAV1.csv'")
except Exception as e:
    print(f"Error loading NAV: {e}")


=== NAV DATA ANALYSIS ===

NAV Data Shape: (223, 7)

NAV Columns:
['Remark', 'QB Num', 'No.', 'Item', 'Qty(+)', 'Ship Date', 'Customer Ordering Desc.']

NAV Data Types:
Remark                     object
QB Num                     object
No.                        object
Item                       object
Qty(+)                      int64
Ship Date                  object
Customer Ordering Desc.    object
dtype: object

NAV First 3 rows:
       Remark            QB Num               No.                    Item  \
0  LO25080011  For NTA_CoastIPC  950-RS880500-A13              RGS-8805GC   
1  SO25050035        POD-250648  S51-SL1708FF-003  SEMIL-1708-i7OCG9-FF04   
2  SO25050035        POD-250648  395-M1342189-004   Cbl-M12S4F-OW4-180CM1   

   Qty(+)   Ship Date                            Customer Ordering Desc.  
0       1   2025/9/10  Rugged and Compact AMD EPYC 7003/7002 "MILAN/R...  
1      10  2025/11/19  SEMIL-1708-FF, including i7-9700TE, DDR4-16GB-...  
2      10  2025/11/19  Cab

In [134]:
print("=== KEY COLUMNS ANALYSIS ===")
print()

# Analyze key columns for matching
if 'pod' in locals() and 'nav' in locals():
    print("QB Num matching analysis:")
    print(f"POD unique QB Nums: {pod['QB Num'].nunique()}")
    print(f"NAV unique QB Nums: {nav['QB Num'].nunique()}")
    
    # Check for common QB Nums
    pod_qb = set(pod['QB Num'].dropna())
    nav_qb = set(nav['QB Num'].dropna())
    only_in_pod = sorted(pod_qb - nav_qb)   # in POD, not in NAV
    only_in_nav = sorted(nav_qb - pod_qb)   # in NAV, not in POD

    print(f"Only in POD: {len(only_in_pod)}")
    print(f"Only in NAV: {len(only_in_nav)}")
    print(only_in_nav)
    
    # print("\nItem matching analysis:")
    # print(f"POD unique Items: {pod['Item'].nunique()}")
    # print(f"NAV unique Items: {nav['Item'].nunique()}")
    
    # # Check for common Items
    # pod_items = set(pod['Item'].dropna())
    # nav_items = set(nav['Item'].dropna())
    # common_items = pod_items.intersection(nav_items)
    # print(f"Common Items: {len(common_items)}")
    
    print("\nPre-installed items analysis (S-prefix):")
    s_items = nav[nav['No.'].astype(str).str.startswith('S', na=False)]
    print(f"NAV items with S-prefix: {s_items['Item'].nunique()}")
    print(f"Sample S-prefix items: {s_items['Item'].head(5).tolist()}")
    
    # print("\nShip Date analysis:")
    # print(f"NAV unique Ship Dates: {nav['Ship Date'].nunique()}")
    # print(f"Sample Ship Dates: {sorted(nav['Ship Date'].dropna().unique())[:5]}")
else:
    print("Cannot analyze - data not loaded properly")


=== KEY COLUMNS ANALYSIS ===

QB Num matching analysis:
POD unique QB Nums: 154
NAV unique QB Nums: 92
Only in POD: 63
Only in NAV: 1
['For NTA_CoastIPC']

Pre-installed items analysis (S-prefix):
NAV items with S-prefix: 23
Sample S-prefix items: ['SEMIL-1708-i7OCG9-FF04', 'SEMIL-1708-i7OCG9-FF04', 'SEMIL-1708-i7OCG9-FF04', 'POC-330-AEI01-100', 'POC-330-AEI01-100']


In [135]:
nav[nav['QB Num'] == 'For NTA_CoastIPC']

,Remark,QB Num,No.,Item,Qty(+),Ship Date,Customer Ordering Desc.
0,LO25080011,For NTA_CoastIPC,950-RS880500-A13,RGS-8805GC,1,2025/9/10,"Rugged and Compact AMD EPYC 7003/7002 ""MILAN/R..."


## NAV Analysis

In [136]:
# Ensure we have the data loaded
if 'pod' not in locals() or 'nav' not in locals():
    print("Loading data first...")
    pod = pd.read_csv('open purchase2.csv')
    nav = pd.read_csv('NAV1.csv')

# Debug: Check NAV columns
print("NAV columns:", nav.columns.tolist())
print("pod columns:", pod.columns.tolist())
# print("NAV sample data:")
# print(nav.head(2))

# # Create a copy of POD for processing
# enhanced_pod = pod.copy()

# # Add new columns
# enhanced_pod['Barebone/Pre-installed'] = ''
# enhanced_pod['Ship Date'] = ''
# enhanced_pod['Status'] = ''

print(f"\nStarting with {len(pod)} POD records")
print(f"\nNAV has {len(nav)} records")



# Check if Remark column contains S-prefix information
Pre_NAV = nav[nav['No.'].astype(str).str.startswith('S', na=False)]
mask_not_s = ~nav['No.'].astype(str).str.startswith('S', na=False)
Bare_NAV  = nav[mask_not_s]
print(f"\nFound {len(Pre_NAV)} pre-installed items in NAV ")
print(f"\nFound {len(Bare_NAV)} Barebone items in NAV ")
print(f"\n\n\nSample pre-installed items: \n{Pre_NAV.head}")



NAV columns: ['Remark', 'QB Num', 'No.', 'Item', 'Qty(+)', 'Ship Date', 'Customer Ordering Desc.']
pod columns: ['Order Date', 'QB Num', 'P. O. #', 'Name', 'Deliv Date', 'Qty(+)', 'Item', 'Inventory Site']

Starting with 459 POD records

NAV has 223 records

Found 37 pre-installed items in NAV 

Found 186 Barebone items in NAV 



Sample pre-installed items: 
<bound method NDFrame.head of          Remark      QB Num               No.                           Item  \
1    SO25050035  POD-250648  S51-SL1708FF-003         SEMIL-1708-i7OCG9-FF04   
3    SO25050036  POD-250649  S51-SL1708FF-003         SEMIL-1708-i7OCG9-FF04   
5    SO25050037  POD-250650  S51-SL1708FF-003         SEMIL-1708-i7OCG9-FF04   
7    SO25070419  POD-251059  S60-UC033000-001              POC-330-AEI01-100   
9    SO25070420  POD-251060  S60-UC033000-001              POC-330-AEI01-100   
11   SO25070496  POD-251062  S51-UC0715CG-001        POC-715-CGN1-CF2-25(EA)   
14   SO25070496  POD-251062  S51-UC0715C1-001   

### Partical Shipment

In [137]:
import pandas as pd

# 1) Normalize
NAVc = nav.copy()
for c in ['QB Num','Item']:
    NAVc[c] = NAVc[c].astype(str).str.replace('\u00A0',' ', regex=False).str.strip()
NAVc['Ship Date'] = pd.to_datetime(NAVc['Ship Date'], errors='coerce')

# 2) Group and count distinct dates per (QB Num, Item)
agg = (NAVc.groupby(['QB Num','Item'], as_index=False)
          .agg(n_rows=('Ship Date','size'),
               n_unique_dates=('Ship Date','nunique'),
               dates=('Ship Date', lambda s: sorted(s.dropna().dt.strftime('%Y/%m/%d').unique()))))

# 3) Filter to those with >1 distinct Ship Date
multi_date_pairs = agg[agg['n_unique_dates'] > 1]

# Results
num_pairs = len(multi_date_pairs)                       # how many (QB Num, Item) pairs
rows_involved = (NAVc
                 .merge(multi_date_pairs[['QB Num','Item']], on=['QB Num','Item'])
                 .shape[0])                             # how many NAV rows in those pairs

print(f"Pairs with multiple Ship Dates: {num_pairs}")
print(f"Rows involved (all occurrences of those pairs): {rows_involved}")
print(multi_date_pairs.head(10).to_string(index=False))  # preview


Pairs with multiple Ship Dates: 2
Rows involved (all occurrences of those pairs): 4
    QB Num                                Item  n_rows  n_unique_dates                    dates
POD-251182 Cbl-M12A5F-OT2-Black-Red-Fuse-100CM       2               2 [2025/10/16, 2025/10/29]
POD-251244              mPCIeHS-BTWifi-WT-6218       2               2 [2025/10/29, 2025/12/24]


In [138]:
import re
import pandas as pd
import numpy as np

# Include the label in your desired column order
COLS = ['Remark','QB Num','No.','Item','Qty(+)','Ship Date','Customer Ordering Desc.','Pre/Bare']

def explode_pre_nav(pre_nav: pd.DataFrame, keep_base: bool = True) -> pd.DataFrame:
    pre = pre_nav.fillna('').copy()
    rows = []
    for rec in pre.to_dict('records'):
        desc = (str(rec.get('Customer Ordering Desc.', ''))
                .replace('\u00A0',' ').replace('\u3000',' ').strip())
        parts = re.split(r',\s*including\s*', desc, maxsplit=1, flags=re.I)
        base  = parts[0].split(',', 1)[0].strip()
        comps = [c.strip() for c in parts[1].split(',')] if len(parts) > 1 else []

        targets, seen = [], set()
        for t in ([*comps, base] if keep_base else comps):
            key = t.lower()
            if t and key not in seen:
                seen.add(key); targets.append(t)

        for t in targets:
            new = rec.copy()
            new['Customer Ordering Desc.'] = t
            compact = t.replace(' ', '')
            m = re.match(r'^(\d+)x(.+)$', compact, flags=re.I)
            if m:
                mult = int(m.group(1))
                new['Item'] = m.group(2)
                qty = pd.to_numeric(new.get('Qty(+)'), errors='coerce')
                new['Qty(+)'] = (float(qty) if pd.notna(qty) else 1.0) * mult
            else:
                new['Item'] = t
            rows.append(new)

    out = pd.DataFrame(rows)
    return out

# --- Build Final_NAV with labels ---
Expanded_Pre = explode_pre_nav(Pre_NAV, keep_base=True).copy()
Expanded_Pre['Pre/Bare'] = 'Pre'

Bare_NAV_labeled = Bare_NAV.copy()
Bare_NAV_labeled['Pre/Bare'] = 'Bare'

stacked = pd.concat([Expanded_Pre, Bare_NAV_labeled], ignore_index=True)

# hygiene / optional mapping
stacked['Qty(+)'] = pd.to_numeric(stacked['Qty(+)'], errors='coerce')
try:
    stacked['Item'] = stacked['Item'].replace(NAV_mappings)
except NameError:
    pass

# Prefer Pre over Bare on duplicates across these keys
KEYS = ['Remark','QB Num','No.','Item','Ship Date','Customer Ordering Desc.']
stacked['__rank'] = stacked['Pre/Bare'].map({'Pre': 0, 'Bare': 1})
Final_NAV = (stacked.sort_values('__rank')
                   .drop(columns='__rank')
                   .reset_index(drop=True))

# Keep the label column (don’t drop it via reindex)
Final_NAV = Final_NAV[[c for c in COLS if c in Final_NAV.columns]]

print(f"Pre_NAV rows: {len(Pre_NAV)}  -> Expanded: {len(Expanded_Pre)} | "
      f"Bare rows: {len(Bare_NAV_labeled)} | Final rows: {len(Final_NAV)}")

Final_NAV


Pre_NAV rows: 37  -> Expanded: 184 | Bare rows: 186 | Final rows: 370


,Remark,QB Num,No.,Item,Qty(+),Ship Date,Customer Ordering Desc.,Pre/Bare
0,SO25050035,POD-250648,S51-SL1708FF-003,i7-9700TE,10.0,2025/11/19,i7-9700TE,Pre
1,SO25090098,POD-251261,S51-SL2047CR-006,M.280-SSD-4TB-PCIe4-TLCWT5NH-IK,3.0,2025/10/22,M.280-SSD-4TB-PCIe4-TLCWT5-NH-IK,Pre
2,SO25090098,POD-251261,S51-SL2047CR-006,DDR5-32GB-48WT-SM,6.0,2025/10/22,2x DDR5-32GB-48WT-SM,Pre
3,SO25090098,POD-251261,S51-SL2047CR-006,mPCIe-COM-4RS232/422/485-X404,3.0,2025/10/22,mPCIe-COM-4RS232/422/485-X404,Pre
4,SO25090098,POD-251261,S51-SL2047CR-006,mPCIe-CAN-IPEH-4047,3.0,2025/10/22,mPCIe-CAN-IPEH-4047,Pre
...,...,...,...,...,...,...,...,...
365,SO25090021,POD-251251,395-M1452189-004,Cbl-M12L5F-CordEnd5-180CM,4.0,2025/10/8,"M12 L-Code 5P(FML) to Cord End Terminal 5P, Le...",Bare
366,SO25090021,POD-251251,395-M1182181-104,Cbl-M12A8M-2U2TA-180CM2,4.0,2025/10/8,Waterproof M12 (8-pole-A-coded) w/ copper shie...,Bare
367,SO25090072,POD-251255,950-EC900232-A21,Nuvo-9002LP,4.0,2025/10/16,Intel® 14th/ 13th/12th -Gen Core™ Rugged Embed...,Bare
368,SO25080505,POD-251244,501-W1336218-077,mPCIeHS-BTWifi-WT-6218,2.0,2025/12/24,"802.11ac/a/b/g/n, 2T2R WiFi + BT4.2 half size ...",Bare


In [144]:
import numpy as np
import pandas as pd

PARTIAL_LABEL = "Partical Shipement"  

# --- normalize ---
NAVc = nav.copy()
PODc = pod.copy()

for c in ['QB Num','Item']:
    NAVc[c] = NAVc[c].astype(str).str.replace('\u00A0',' ', regex=False).str.strip()
    PODc[c] = PODc[c].astype(str).str.replace('\u00A0',' ', regex=False).str.strip()

NAVc['Ship Date'] = pd.to_datetime(NAVc['Ship Date'], errors='coerce')

# wipe POD dates first
PODc = PODc.drop(columns=['Deliv Date'], errors='ignore')

# --- per (QB Num, Item): count rows + pick a date (any single date: max works) ---
nav_g = (NAVc.groupby(['QB Num','Item'], as_index=False)
           .agg(Ship_Date_any=('Ship Date','max'),
                n_rows=('Ship Date','size')))

# if multiple rows -> label; else -> date
nav_g['Deliv Date'] = np.where(
    nav_g['n_rows'] > 1,
    PARTIAL_LABEL,
    nav_g['Ship_Date_any'].dt.strftime('%Y/%m/%d')
)

# keep only the columns we need to merge
lookup = nav_g[['QB Num','Item','Deliv Date']]

# --- merge into POD ---
PODc = PODc.merge(lookup, on=['QB Num','Item'], how='left')

# (optional) quick counts
num_partial = (PODc['Deliv Date'].astype(str).str.strip() == PARTIAL_LABEL).sum()
num_na = PODc['Deliv Date'].isna().sum()
print(f"Partial rows: {num_partial}  |  NA Deliv Date: {num_na}")




PODc


Partial rows: 2  |  NA Deliv Date: 318


,Order Date,QB Num,P. O. #,Name,Qty(+),Item,Inventory Site,Deliv Date
0,2025/07/24,POD-251046,SO-20251050,Neousys Technology Incorp.,34.0,AccsyBx-6AntiVG-POC-551VTC,Drop Ship,NaN
1,2025/07/24,POD-251047,SO-20251051,Neousys Technology Incorp.,24.0,AccsyBx-6AntiVG-POC-551VTC,Drop Ship,NaN
2,2025/07/24,POD-251048,SO-20251052,Neousys Technology Incorp.,29.0,AccsyBx-6AntiVG-POC-551VTC,Drop Ship,NaN
3,2025/09/18,POD-251338,SO-20251316,Neousys Technology Incorp.,1.0,AccsyBx-Cardholder-9160GC-2000E,WH01S-NTA,NaN
4,2025/09/08,POD-251279,SO-20251262,Neousys Technology Incorp.,8.0,AccsyBx-FAN-NRU-100,WH01S-NTA,2025/10/08
...,...,...,...,...,...,...,...,...
454,2025/09/11,POD-251301,SO-20251281,Neousys Technology Incorp.,1.0,Win10Pro-ENG64,WH01S-NTA,2025/10/22
455,2025/09/12,POD-251305,SO-20251295,Neousys Technology Incorp.,1.0,Win10Pro-ENG64,WH01S-NTA,2025/10/22
456,2025/09/17,POD-251333,SO-20251311,Neousys Technology Incorp.,10.0,Win11IoT24-Entry,WH01S-NTA,2025/10/22
457,2025/09/09,POD-251293,SO-20251285,Neousys Technology Incorp.,1.0,nan,NaN,NaN


In [ ]:
# Count missing Deliv Date (NaN / None / blanks / "NA"-like)
col = PODc['Deliv Date']

mask_na = (
    col.isna() |
    col.astype(str).str.strip().eq('') |
    col.astype(str).str.lower().isin(['na', 'n/a', 'nan', 'none'])
)
num_na = mask_na.sum()



Drop_Ship = PODc[PODc['Inventory Site'] == 'Drop Ship']

Local_Vendor = PODc[PODc['Name'] != 'Neousys Technology Incorp.']

print(f"NA Deliv Date: {num_na}")
print(f"Drop Ship: {len(Drop_Ship)}")
print(f"Local Vendor: {len(Local_Vendor)}")

NA Deliv Date: 318
Drop Ship: 83
Local Vendor: 25
Partial: 1


In [147]:
pod_nav = PODc.loc[PODc['Inventory Site'].astype(str).str.strip().str.casefold() != 'drop ship'].copy()
pod_nav.to_csv('pod_nav.csv', index=False)
pod_nav

,Order Date,QB Num,P. O. #,Name,Qty(+),Item,Inventory Site,Deliv Date
3,2025/09/18,POD-251338,SO-20251316,Neousys Technology Incorp.,1.0,AccsyBx-Cardholder-9160GC-2000E,WH01S-NTA,NaN
4,2025/09/08,POD-251279,SO-20251262,Neousys Technology Incorp.,8.0,AccsyBx-FAN-NRU-100,WH01S-NTA,2025/10/08
7,2025/09/17,POD-251333,SO-20251311,Neousys Technology Incorp.,20.0,Ant-RP_SMAM-WiFi-108MM,WH01S-NTA,2025/10/22
8,2025/09/30,POD-251383,NaN,Neousys Technology Incorp.,20.0,Ant-RP_SMAM-WiFi-108MM,WH01S-NTA,NaN
9,2025/08/15,POD-251164,SO-20251167,Neousys Technology Incorp.,2.0,Ant-RP_SMAM-WiFi-196MM1,WH01S-NTA,2025/10/16
...,...,...,...,...,...,...,...,...
454,2025/09/11,POD-251301,SO-20251281,Neousys Technology Incorp.,1.0,Win10Pro-ENG64,WH01S-NTA,2025/10/22
455,2025/09/12,POD-251305,SO-20251295,Neousys Technology Incorp.,1.0,Win10Pro-ENG64,WH01S-NTA,2025/10/22
456,2025/09/17,POD-251333,SO-20251311,Neousys Technology Incorp.,10.0,Win11IoT24-Entry,WH01S-NTA,2025/10/22
457,2025/09/09,POD-251293,SO-20251285,Neousys Technology Incorp.,1.0,nan,NaN,NaN


In [143]:
df = PODc.loc[mask_na & (PODc['Inventory Site'].astype(str).str.strip().str.casefold() != 'drop ship')].copy()
df = df.assign(
    _po_num = df['QB Num'].astype(str).str.extract(r'(\d+)', expand=False).astype(float)
)
df.sort_values(by=['QB Num'], na_position='last') \
  .drop(columns=['_po_num']) \
  .to_csv('POD_No Deliv_Date.csv', index=False)


### IF 'Pre/Bare' is NA, then the row is only in POD, not in NAV yet
### IF 'Pre/Bare' is no NA, but 'Deliv Date' is NA, then the row is in NAV while "OP Estimated Shipping Date" is NA in NAV